In [1]:
# import libraries
try:
  # Uninstall tf-nightly and install a stable TensorFlow version
  #!pip uninstall -y tf-nightly tensorflow
  !pip install tensorflow
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

/usr/local/lib/python3.12/dist-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


2.20.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-12-23 04:12:57--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.5’

train-data.tsv.5    100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2025-12-23 04:12:57 (69.8 MB/s) - ‘train-data.tsv.5’ saved [358233/358233]

--2025-12-23 04:12:58--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.5’

valid-data.tsv.5    100%[========

In [3]:
from tensorflow.keras import layers

In [4]:
import pandas as pd

train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
print(train_df.head())

  label                                            message
0   ham  ahhhh...just woken up!had a bad dream about u ...
1   ham                           you can never do nothing
2   ham  now u sound like manky scouse boy steve,like! ...
3   ham  mum say we wan to go then go... then she can s...
4   ham  never y lei... i v lazy... got wat? dat day ü ...


In [5]:
# Extract messages
train_messages = train_df['message'].values
test_messages = test_df['message'].values

# Map 'ham' to 0 and 'spam' to 1 for labels
label_mapping = {'ham': 0, 'spam': 1}
train_labels = np.array([label_mapping[label] for label in train_df['label']])
test_labels = np.array([label_mapping[label] for label in test_df['label']])

#print(train_labels)

In [6]:
#Remove HTML
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
  clean_data  = custom_standardization(train_messages)

In [7]:
#Vectorization
max_features = 10000
sequence_length = 120

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)



In [8]:
import re
import string
vectorize_layer.adapt(train_messages)

In [9]:
# Transform messages to sequences
train_sequences = vectorize_layer(train_messages)
test_sequences = vectorize_layer(test_messages)

print(f"\nVectorization example:")
print(f"Original: {train_messages[0]}")
print(f"Vectorized: {train_sequences[0].numpy()[:20]}...")


Vectorization example:
Original: ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.
Vectorized: [7632 2425 3973    5  404  824   79    7 4156    3   47   57    7  153
   22    3  177   56  165   79]...


In [10]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [30]:
print("\n" + "="*50)
print("Training the model...")
print("="*50)

history = model.fit(
    train_sequences,
    train_labels,
    epochs=60,
    validation_data=(test_sequences, test_labels),
    verbose=1
)


Training the model...
Epoch 1/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9873 - loss: 0.0473 - val_accuracy: 0.9770 - val_loss: 0.0715
Epoch 2/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9878 - loss: 0.0460 - val_accuracy: 0.9756 - val_loss: 0.0673
Epoch 3/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9888 - loss: 0.0426 - val_accuracy: 0.9763 - val_loss: 0.0662
Epoch 4/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9873 - loss: 0.0428 - val_accuracy: 0.9777 - val_loss: 0.0651
Epoch 5/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9897 - loss: 0.0408 - val_accuracy: 0.9777 - val_loss: 0.0645
Epoch 6/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9885 - loss: 0.0403 - val_accuracy: 0.9756 - val_loss: 0.0649
Epoch 7/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9888 - loss: 0.0382 - val_accuracy: 0.9770 - val_loss: 0.0632
Epoch 8/60
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9897 - loss:

In [31]:
# Right after training
test_msg = "you have won £1000 cash! call to claim your prize."
vectorized = vectorize_layer([test_msg])
prediction = model.predict(vectorized, verbose=0)[0][0]
print(f"Direct prediction after training: {prediction}")
print(f"Label: {'spam' if prediction > 0.5 else 'ham'}")

# Check vocabulary
print(f"Vocabulary size: {len(vectorize_layer.get_vocabulary())}")
print(f"First 20 words: {vectorize_layer.get_vocabulary()[:20]}")

Direct prediction after training: 0.9989098310470581
Label: spam
Vocabulary size: 8181
First 20 words: ['', '[UNK]', np.str_('to'), np.str_('i'), np.str_('you'), np.str_('a'), np.str_('the'), np.str_('u'), np.str_('and'), np.str_('in'), np.str_('is'), np.str_('me'), np.str_('my'), np.str_('for'), np.str_('your'), np.str_('of'), np.str_('it'), np.str_('call'), np.str_('have'), np.str_('on')]


In [32]:
#Evaluate model
print("Training set:")
print(f"Ham: {sum(train_labels == 0)}")
print(f"Spam: {sum(train_labels == 1)}")
print(f"Spam ratio: {sum(train_labels == 1) / len(train_labels):.2%}")

print("\n" + "="*50)
print("Evaluating on test data...")
print("="*50)

loss, accuracy = model.evaluate(test_sequences, test_labels)
print(f"\nTest Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")

Training set:
Ham: 3619
Spam: 560
Spam ratio: 13.40%

Evaluating on test data...
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9856 - loss: 0.0552

Test Accuracy: 0.9856
Test Loss: 0.0552


In [33]:
#Save model
model.save('spam_model.keras')
print("\nModel saved as 'spam_model.keras'")


Model saved as 'spam_model.keras'


In [34]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Vectorize the input text
  vectorized = vectorize_layer([pred_text])

  # Make prediction
  prediction = model.predict(vectorized, verbose=0)[0][0]

  # Convert to label
  label = 'spam' if prediction > 0.5 else 'ham'


  return [float(prediction), label] # Changed to return a single list

pred_text = "Hi ,how are you today "

prediction = predict_message(pred_text)
print(prediction)

[0.003446837654337287, 'ham']


In [35]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]
  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(f"Message: {msg[:40]}...")
    print(f"  Predicted: {prediction[1]} (prob: {prediction[0]:.4f})")
    print(f"  Expected: {ans}")
    print(f"  Match: {prediction[1] == ans}\n")
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

Message: how are you doing today...
  Predicted: ham (prob: 0.0006)
  Expected: ham
  Match: True

Message: sale today! to stop texts call 989124603...
  Predicted: spam (prob: 0.5105)
  Expected: spam
  Match: True

Message: i dont want to go. can we try it a diffe...
  Predicted: ham (prob: 0.0000)
  Expected: ham
  Match: True

Message: our new mobile video service is live. ju...
  Predicted: spam (prob: 0.9677)
  Expected: spam
  Match: True

Message: you have won £1000 cash! call to claim y...
  Predicted: spam (prob: 0.9989)
  Expected: spam
  Match: True

Message: i'll bring it tomorrow. don't forget the...
  Predicted: ham (prob: 0.0001)
  Expected: ham
  Match: True

Message: wow, is your arm alright. that happened ...
  Predicted: ham (prob: 0.0003)
  Expected: ham
  Match: True

You passed the challenge. Great job!
